In [6]:
import json
import ollama

In [7]:
SYSTEM_PROMPT = ""
with open("../../global_system_prompt.txt", "r", encoding="utf-8") as f:
    SYSTEM_PROMPT = f.read()
VALIDATE_PROMPT = ""
with open("../../validity_check_prompt.txt", "r", encoding="utf-8") as f:
    VALIDATE_PROMPT = f.read()

In [8]:
from collections import Counter

def get_structure(json_data):
    try:
        data = json.loads(json_data)
        return {
            "atoms_count": len(data.get("atoms", [])),
            "rules_count": len(data.get("rules", [])),
            "strict_rules": len([r for r in data.get("rules", []) if r.get("type") == "strict"]),
            "defeasible_rules": len([r for r in data.get("rules", []) if r.get("type") == "defeasible"]),
            "attacks_count": len(data.get("attacks", [])),
            "attack_types": Counter([a.get("type") for a in data.get("attacks", [])])
        }
    except:
        return "Invalid JSON"

In [12]:
prompts=[]
with open("prompts.txt", "r") as f:
    prompts = f.read()
for i in prompts.split("@"):
    print("Prompt:", i)
    response = ollama.chat(
        model="qwen2.5:14b",
        format="json",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": i}
        ],
        options={"temperature": 0}
    )
    print("Primary model response:\n",response["message"]["content"].strip())
    response = ollama.chat(
        model="qwen2.5:14b",
        format="json",
        messages=[
            {"role": "system", "content": VALIDATE_PROMPT+i},
            {"role": "user", "content": response["message"]["content"].strip()}
        ],
        options={"temperature": 0}
    )
    print("Validation response:\n",response["message"]["content"].strip())
    
    

Prompt: If a creature is a bird, it has wings.  
This creature has wings.  
Therefore, this creature is a bird.

Primary model response:
 {
  "atoms": [
    {"id": "a1", "text": "If a creature is a bird, it has wings"},
    {"id": "a2", "text": "This creature has wings"},
    {"id": "a3", "text": "Therefore, this creature is a bird"}
  ],
  "rules": [
    {"id": "r1", "type": "defeasible", "premises": ["a1"], "conclusion": "a3"}
  ],
  "arguments": [
    {"id": "arg1", "premises": ["a2"], "applied_rules": ["r1"], "conclusion": "a3", "sub_arguments": []}
  ],
  "attacks": []
}
Validation response:
 {
  "atoms": [
    {"id": "a1", "text": "a creature is a bird"},
    {"id": "a2", "text": "it has wings"},
    {"id": "a3", "text": "This creature has wings"}
  ],
  "rules": [
    {"id": "r1", "type": "defeasible", "premises": ["a1"], "conclusion": "a2"}
  ],
  "arguments": [
    {"id": "arg1", "premises": ["a3"], "applied_rules": [], "conclusion": "a3", "sub_arguments": []}
  ],
  "attacks"